<a href="https://colab.research.google.com/github/JulioSan02/UNCC/blob/4105/HW5/JS_4105_HW5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Data/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def data_map(x):
    return x.map({'yes': 1, 'no': 0, 'unfurnished': 0, 'semi-furnished': 0.5, 'furnished': 1})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(data_map)
print('All housing data with binary inputs replaced with ones and zeros: \n')
housing.head()

All housing data with binary inputs replaced with ones and zeros: 



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1.0,0.0,0.0,0.0,1.0,2,1.0,1.0
1,12250000,8960,4,4,4,1.0,0.0,0.0,0.0,1.0,3,0.0,1.0
2,12250000,9960,3,2,2,1.0,0.0,1.0,0.0,0.0,2,1.0,0.5
3,12215000,7500,4,2,2,1.0,0.0,1.0,0.0,1.0,3,1.0,1.0
4,11410000,7420,4,1,2,1.0,1.0,1.0,0.0,1.0,2,0.0,1.0


In [4]:
np.random.seed(0)
train, test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 100)

scale = StandardScaler()

price_train_og = train.values[:, 0]
price_train_t = torch.tensor(price_train_og)

price_test_og = test.values[:, 0]
price_test_t = torch.tensor(price_test_og)

train_len = len(price_train_og)  # Number of training examples for training set
test_len = len(price_test_og)  # Number of training examples for test set

vars_all  = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

train_stand_og = train
train_stand_og[vars_all] = scale.fit_transform(train[vars_all])

test_stand_og = test
test_stand_og[vars_all] = scale.fit_transform(test[vars_all])

# Inputs using all variables
vars_5_train_stand_og = np.c_[np.ones((train_len, 1)), train_stand_og[vars_all]]
vars_5_train_stand = torch.tensor(vars_5_train_stand_og)

vars_5_test_stand_og = np.c_[np.ones((test_len, 1)), test_stand_og[vars_all]]
vars_5_test_stand = torch.tensor(vars_5_test_stand_og)


params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)


In [5]:
def model(t_in, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, b):
  t_p = (w12 * t_in ** 12) + (w11 * t_in ** 11) + (w10 * t_in ** 10) + (w9 * t_in ** 9) + (w8 * t_in ** 8) + (w7 * t_in ** 7) + (w6 * t_in ** 6) + (w5 * t_in ** 5) + (w4 * t_in ** 4) + (w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b
  return t_p

def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, test_t_in,
                    train_t_out, test_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model(train_t_in, *params)
    train_loss = loss_fn(train_t_p.transpose(0,1), train_t_out)

    test_t_p = model(test_t_in, *params)
    test_loss = loss_fn(test_t_p.transpose(0,1), test_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {test_loss.item():.4f}")

  return params

In [6]:
learning_rate_1 = 1e-14
optimizer_1 = optim.SGD([params_og], lr=learning_rate_1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 116104662602546.2188, 	Validation loss 2015330259410519851008.0000
Epoch 3, 	Training loss 89138324928940.6406, 	Validation loss 1003097254716660252672.0000
Epoch 500, 	Training loss 26298619610665.0742, 	Validation loss 17276544265535645696.0000
Epoch 1000, 	Training loss 26289232776943.3438, 	Validation loss 25102748551228768256.0000
Epoch 1500, 	Training loss 26281786589331.3555, 	Validation loss 33285609883020902400.0000
Epoch 2000, 	Training loss 26275878097504.1680, 	Validation loss 41527901062902128640.0000
Epoch 2500, 	Training loss 26271187973922.1016, 	Validation loss 49618857595260895232.0000
Epoch 3000, 	Training loss 26267463422894.8359, 	Validation loss 57412812983561568256.0000
Epoch 3500, 	Training loss 26264503736239.0000, 	Validation loss 64814825565215653888.0000
Epoch 4000, 	Training loss 26262150114189.8047, 	Validation loss 71766511975299006464.0000
E

tensor([ 1.0000e+00,  1.0006e+00,  1.0006e+00,  1.0016e+00,  1.0027e+00,
         1.0098e+00,  1.0233e+00,  1.0843e+00,  1.2189e+00,  1.6658e+00,
         2.0810e+00, -4.3456e-01,  4.7709e-04], requires_grad=True)

In [7]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_2 = 1e-15
optimizer_2 = optim.SGD([params_og], lr=learning_rate_2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_2,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 111836777523794.9219, 	Validation loss 1405533816648371798016.0000
Epoch 3, 	Training loss 83307888353362.0156, 	Validation loss 901068437951574835200.0000
Epoch 500, 	Training loss 26309121681674.5586, 	Validation loss 10885770138183544832.0000
Epoch 1000, 	Training loss 26307841512475.9219, 	Validation loss 11555270475543707648.0000
Epoch 1500, 	Training loss 26306590719851.4609, 	Validation loss 12236569680968081408.0000
Epoch 2000, 	Training loss 26305368579430.0625, 	Validation loss 12929125407180994560.0000
Epoch 2500, 	Training loss 26304174451083.7539, 	Validation loss 13632331866142291968.0000
Epoch 3000, 	Training loss 26303007659726.7031, 	Validation loss 14345632924515481600.0000
Epoch 3500, 	Training loss 26301867687524.8008, 	Validation loss 15068440462883371008.0000
Epoch 4000, 	Training loss 26300753862436.7539, 	Validation loss 15800314739688769536.0000
Ep

tensor([ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0006e+00,
         1.0011e+00,  1.0029e+00,  1.0103e+00,  1.0260e+00,  1.0640e+00,
         9.9124e-01, -2.0488e-01,  4.7763e-05], requires_grad=True)

In [8]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_3 = 1e-16
optimizer_3 = optim.SGD([params_og], lr=learning_rate_3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_3,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 149974813584831.2500, 	Validation loss 2091117530292320468992.0000
Epoch 3, 	Training loss 145474392623382.9062, 	Validation loss 2009745312662845915136.0000
Epoch 500, 	Training loss 26310298308392.6953, 	Validation loss 10264269618284945408.0000
Epoch 1000, 	Training loss 26310165184425.5352, 	Validation loss 10360258639608942592.0000
Epoch 1500, 	Training loss 26310033577712.2266, 	Validation loss 10425588110579992576.0000
Epoch 2000, 	Training loss 26309902123663.0508, 	Validation loss 10491118842396545024.0000
Epoch 2500, 	Training loss 26309770822000.4336, 	Validation loss 10556858960899532800.0000
Epoch 3000, 	Training loss 26309639673001.9492, 	Validation loss 10622800314789365760.0000
Epoch 3500, 	Training loss 26309508676390.0352, 	Validation loss 10688951080824285184.0000
Epoch 4000, 	Training loss 26309378218716.4102, 	Validation loss 10755215114651293696.0000


tensor([ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  9.9999e-01,
         9.9995e-01,  1.0000e+00,  9.9966e-01,  9.9559e-01,  9.6920e-01,
         8.0227e-01, -1.6542e-01,  4.7733e-06], requires_grad=True)

In [9]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_4 = 1e-17
optimizer_4 = optim.SGD([params_og], lr=learning_rate_4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_4,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 154174268014350.0312, 	Validation loss 2167141570169519996928.0000
Epoch 3, 	Training loss 153705067272205.6250, 	Validation loss 2158643143742955192320.0000
Epoch 500, 	Training loss 46804915381794.9609, 	Validation loss 279511346464365248512.0000
Epoch 1000, 	Training loss 29571271247991.3164, 	Validation loss 22510751015875358720.0000
Epoch 1500, 	Training loss 26829225356821.9570, 	Validation loss 1070511573749742.5000
Epoch 2000, 	Training loss 26392928447327.0703, 	Validation loss 3757301895460858368.0000
Epoch 2500, 	Training loss 26323497996457.8477, 	Validation loss 7286093917813527552.0000
Epoch 3000, 	Training loss 26312439742572.5547, 	Validation loss 9020412393714788352.0000
Epoch 3500, 	Training loss 26310669085209.9258, 	Validation loss 9767664212581189632.0000
Epoch 4000, 	Training loss 26310376227679.0859, 	Validation loss 10077883683796547584.0000
Epoch 4

tensor([ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         9.9996e-01,  9.9973e-01,  9.9855e-01,  9.9237e-01,  9.5928e-01,
         7.8225e-01, -1.6112e-01,  4.7385e-07], requires_grad=True)

In [10]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_5 = 0.0001
optimizer_5 = optim.Adam([params_og], lr=learning_rate_5)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_5,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 154618895854764.5938, 	Validation loss 2175237252952330076160.0000
Epoch 3, 	Training loss 154592594686405.2812, 	Validation loss 2174802111000409800704.0000
Epoch 500, 	Training loss 141997475512469.2188, 	Validation loss 1966286874195732463616.0000
Epoch 1000, 	Training loss 130245161934779.4531, 	Validation loss 1771456464450664529920.0000
Epoch 1500, 	Training loss 119337238030623.8438, 	Validation loss 1590352780266758471680.0000
Epoch 2000, 	Training loss 109209346995387.1406, 	Validation loss 1421923482034225020928.0000
Epoch 2500, 	Training loss 99809254753505.5312, 	Validation loss 1265314584803263119360.0000
Epoch 3000, 	Training loss 91095151620826.1094, 	Validation loss 1119843044048738713600.0000
Epoch 3500, 	Training loss 83033656980540.3906, 	Validation loss 984964182246442663936.0000
Epoch 4000, 	Training loss 75598186258735.3594, 	Validation loss 860245535

tensor([0.6524, 0.7421, 0.5522, 0.5485, 0.5448, 0.5441, 0.5433, 0.5430, 0.5427,
        0.5425, 0.5424, 0.5422, 0.5025], requires_grad=True)

In [11]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_6 = 0.001
optimizer_6 = optim.Adam([params_og], lr=learning_rate_6)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_6,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 154382329438557.0312, 	Validation loss 2171323319697049124864.0000
Epoch 3, 	Training loss 154119728831126.1875, 	Validation loss 2166978552205735624704.0000
Epoch 500, 	Training loss 65449742411386.4922, 	Validation loss 689381003198737547264.0000
Epoch 1000, 	Training loss 34200838367445.7461, 	Validation loss 153060863565971259392.0000
Epoch 1500, 	Training loss 27242571136513.4609, 	Validation loss 23411156156313243648.0000
Epoch 2000, 	Training loss 26434453953894.7070, 	Validation loss 3728192513158043648.0000
Epoch 2500, 	Training loss 26393936645504.8516, 	Validation loss 1448401136013681152.0000
Epoch 3000, 	Training loss 26392286007181.5625, 	Validation loss 1155836807781523968.0000
Epoch 3500, 	Training loss 26391016399752.5742, 	Validation loss 1056443570369215360.0000
Epoch 4000, 	Training loss 26389399498074.8945, 	Validation loss 951140209114777344.0000
Epoc

tensor([5.6094, 5.9091, 2.9781, 2.3019, 1.1536, 0.8478, 0.4926, 0.3510, 0.2110,
        0.1304, 0.0605, 0.0122, 5.0451], requires_grad=True)

In [12]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_7 = 0.01
optimizer_7 = optim.Adam([params_og], lr=learning_rate_7)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_7,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 152028601234652.5000, 	Validation loss 2132376543521006157824.0000
Epoch 3, 	Training loss 149439692670558.4375, 	Validation loss 2089527729770179067904.0000
Epoch 500, 	Training loss 26390348222134.0859, 	Validation loss 1014194594416148352.0000
Epoch 1000, 	Training loss 26380383513121.9531, 	Validation loss 464570022264173952.0000
Epoch 1500, 	Training loss 26366698962113.2578, 	Validation loss 46063881019069816.0000
Epoch 2000, 	Training loss 26349889035804.5430, 	Validation loss 154941957418017664.0000
Epoch 2500, 	Training loss 26330701905318.3555, 	Validation loss 1306720119385735168.0000
Epoch 3000, 	Training loss 26310197899557.9297, 	Validation loss 4111174306145554432.0000
Epoch 3500, 	Training loss 26289763541713.0508, 	Validation loss 9169112369118396416.0000
Epoch 4000, 	Training loss 26270966206491.2227, 	Validation loss 16870305581752799232.0000
Epoch 4500,

tensor([50.7667, 51.3029, 41.8997, 37.4371, 24.2127, 19.3165, 11.0435,  7.9306,
         4.2682,  2.4604,  0.7846, -0.2511, 50.0930], requires_grad=True)

In [13]:
params_og = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate_8 = 0.1
optimizer_8 = optim.Adam([params_og], lr=learning_rate_8)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_8,
    params = params_og,
    train_t_in = vars_5_train_stand,
    test_t_in = vars_5_test_stand,
    train_t_out = price_train_t,
    test_t_out = price_test_t)

Epoch 1, 	Training loss 154645199720836.8750, 	Validation loss 2175672438432251772928.0000
Epoch 2, 	Training loss 129692752473038.5625, 	Validation loss 1762294086814668947456.0000
Epoch 3, 	Training loss 107526432497775.0000, 	Validation loss 1393915745358641627136.0000
Epoch 500, 	Training loss 26298428078451.8828, 	Validation loss 6535473105238055936.0000
Epoch 1000, 	Training loss 26238684937827.0469, 	Validation loss 41587825470693531648.0000
Epoch 1500, 	Training loss 26223450385988.0859, 	Validation loss 65283610602647207936.0000
Epoch 2000, 	Training loss 26215845221718.6836, 	Validation loss 65214063781577162752.0000
Epoch 2500, 	Training loss 26207325521068.9062, 	Validation loss 54782380376436776960.0000
Epoch 3000, 	Training loss 26197149225449.6133, 	Validation loss 41379487960006057984.0000
Epoch 3500, 	Training loss 26185131931011.2344, 	Validation loss 27082159115318337536.0000
Epoch 4000, 	Training loss 26171107700780.2383, 	Validation loss 13499471562696652800.0000
E

tensor([ 4.8868e+02,  4.9225e+02,  4.4754e+02,  4.2169e+02,  2.8140e+02,
         2.2595e+02,  7.4202e+01,  5.2058e+01,  6.6371e+00,  4.6686e+00,
        -2.6171e+00,  2.0966e-01,  4.9962e+02], requires_grad=True)